# <font class='ign_color'>5天学ROS</font>

# Unit 4: ROS 行为

<img src="img/drone.jpg" width="450" />

<b>预计完成时间:</b> 2.5 hours
<br><br>
<b>从本单元你将学到：</b>

* 如何创建一个行为服务器？
* 如何构建自己的行为消息？

## Part 2

在前面的课程中，你学习了如何创建一个行为客户端来<b>调用</b> 一个行为服务器。在这一节的课程中，你将学习如何<b>创建</b>你自己的行为服务器。

<figure>
  <img id="fig-4.5" src="img/action_interface.png"/>
  <br>
   <center> <figcaption>图.4.5 - 行为界面图 2</figcaption></center>
</figure>

## 编写一个行为服务器

<p style="background:#EE9023;color:white;">**练习 4.11: 使用Noteb测试 Fibonacci 行为服务器**</p><br>
点击IPython notebook右上角的“play”按钮运行下面的Python代码。
<br><br> 
<div class='white_bg'><img src="img/font-awesome_step-forward.png" style="float:left"/><br><br></div>
<br>
你也可以按下 **<i>[CTRL]+[Enter]</i>** 组合键运行它。<br>
<br>

当程序完成后，别忘了 <span class="ign_red">重启内核</span>。该操作将清除所有ROS通过该python程序生成的节点。这是很必要的，因为python程序只能启动<b>一个</b> rospy 节点。由于该notebook只是一个分割的python脚本，如果你尝试在不重新启动内核的情况下连续执行两个程序，这将给出rospy Exception 错误。
你可以按下该图标执行它。
<br><br>
<div class='white_bg'><img src="img/refresh_icon.png" style="float:left"/><br><br></div>
<br>


下面是一个ROS行为服务器例子的代码。被调用时，该行为服务器将生成一个给定阶数的Fibonacci序列。行为服务器的目标消息必须指定要被计算的序列的阶数，序列被计算时的反馈和最终Fibonacci序列的结果。

<p style="background:#EE9023;color:white;">**练习 4.11结束**</p>

<p style="background:#3B8F10;color:white;" id="prg-4.11a">**Python Program {4.11a}: fibonacci_action_server.py** </p>

In [1]:
#! /usr/bin/env python
import rospy

import actionlib

from actionlib_tutorials.msg import FibonacciFeedback, FibonacciResult, FibonacciAction

class FibonacciClass(object):
    
  # create messages that are used to publish feedback/result
  _feedback = FibonacciFeedback()
  _result   = FibonacciResult()

  def __init__(self):
    # creates the action server
    self._as = actionlib.SimpleActionServer("fibonacci_as", FibonacciAction, self.goal_callback, False)
    self._as.start()
    
  def goal_callback(self, goal):
    # this callback is called when the action server is called.
    # this is the function that computes the Fibonacci sequence
    # and returns the sequence to the node that called the action server
    
    # helper variables
    r = rospy.Rate(1)
    success = True
    
    # append the seeds for the fibonacci sequence
    self._feedback.sequence = []
    self._feedback.sequence.append(0)
    self._feedback.sequence.append(1)
    
    # publish info to the console for the user
    rospy.loginfo('"fibonacci_as": Executing, creating fibonacci sequence of order %i with seeds %i, %i' % ( goal.order, self._feedback.sequence[0], self._feedback.sequence[1]))
    
    # starts calculating the Fibonacci sequence
    fibonacciOrder = goal.order
    for i in xrange(1, fibonacciOrder):
    
      # check that preempt (cancelation) has not been requested by the action client
      if self._as.is_preempt_requested():
        rospy.loginfo('The goal has been cancelled/preempted')
        # the following line, sets the client in preempted state (goal cancelled)
        self._as.set_preempted()
        success = False
        # we end the calculation of the Fibonacci sequence
        break
      
      # builds the next feedback msg to be sent
      self._feedback.sequence.append(self._feedback.sequence[i] + self._feedback.sequence[i-1])
      # publish the feedback
      self._as.publish_feedback(self._feedback)
      # the sequence is computed at 1 Hz frequency
      r.sleep()
    
    # at this point, either the goal has been achieved (success==true)
    # or the client preempted the goal (success==false)
    # If success, then we publish the final result
    # If not success, we do not publish anything in the result
    if success:
      self._result.sequence = self._feedback.sequence
      rospy.loginfo('Succeeded calculating the Fibonacci of order %i' % fibonacciOrder )
      self._as.set_succeeded(self._result)
      
if __name__ == '__main__':
  rospy.init_node('fibonacci')
  FibonacciClass()
  rospy.spin()

<p style="background:#3B8F10;color:white;">**Code Explanation Python Program: {4.11a}**</p>

该例中，行为服务器使用一个名为**<i>Fibonacci.action</i>** 的行为消息定义。该消息由ROS创建，位于**<i>actionlib_tutorials</i>** 功能包中。

In [ ]:
from actionlib_tutorials.msg import FibonacciFeedback, FibonacciResult, FibonacciAction

这里我们导入由该**<i>Fibonacci.action</i>** 文件生成的消息对象。

In [ ]:
_feedback = FibonacciFeedback()
_result   = FibonacciResult()

这里，我们创建将被用来发布该行为的 **feedback** 和 **result** 的消息对象。

In [ ]:
def __init__(self):
    # creates the action server
    self._as = actionlib.SimpleActionServer("fibonacci_as", FibonacciAction, self.goal_callback, False)
    self._as.start()

这是类的构造函数。在构造函数中，我们创建一个名为**"fibonacci_as"** 的行为服务器。该行为服务器使用行为消息 **FibonacciAction** ，有一个名为**goal_callback** 的回调函数。每当有一个新的目标到来时，该行为服务器将被激活。

In [ ]:
def goal_callback(self, goal):
    
    r = rospy.Rate(1)
    success = True

这里定义 **goal callback** 函数。每当一个新的目标发送到该行为服务器，该函数将被调用。

In [ ]:
self._feedback.sequence = []
self._feedback.sequence.append(0)
self._feedback.sequence.append(1)
    
rospy.loginfo('"fibonacci_as": Executing, creating fibonacci sequence of order %i with seeds %i, %i' % ( goal.order, self._feedback.sequence[0], self._feedback.sequence[1]))

这里 **初始化**  Fibonacci 序列，并设置它的初始值（种子）。并且，我们为用户打印该行为服务器将要计算的 Fibonacci序列数据。

In [ ]:
fibonacciOrder = goal.order
for i in xrange(1, fibonacciOrder):

这里启动一个循环，该循环在**goal.order** 值到达前一直运行。很显然，该值是用户从行为服务器发送的Fibonacci序列的阶数。

In [ ]:
if self._as.is_preempt_requested():
    rospy.loginfo('The goal has been cancelled/preempted')
    # the following line, sets the client in preempted state (goal cancelled)
    self._as.set_preempted()
    success = False
    # we end the calculation of the Fibonacci sequence
    break

检查目标是否被取消（preempted）。别忘了，在前面的单元，你已经学习了如何取消一个目标。 

In [ ]:
self._feedback.sequence.append(self._feedback.sequence[i] + self._feedback.sequence[i-1])
self._as.publish_feedback(self._feedback)
r.sleep()

这里真正计算Fibonacci序列的值。你可以从这里查看Fibonacci序列是如何计算的： <a href="https://en.wikipedia.org/wiki/Fibonacci_number" target="_blank">Fibonacci sequence</a>。每当一个新的序列值被计算，我们就发布 **feedback** 。 

In [ ]:
if success:
  self._result.sequence = self._feedback.sequence
  rospy.loginfo('Succeeded calculating the Fibonacci of order %i' % fibonacciOrder )
  self._as.set_succeeded(self._result)

如果一切正常，我们发布 **result** （这是整个Fibonacci序列），同时使用**set_succeeded()** 函数设置该行为为成功（succeeded）。

<p style="background:#3B8F10;color:white;">**End Code Explanation Python Program: {4.11a}**</p>

<p style="background:#EE9023;color:white;">**练习 4.12a: 查看 Fibonacci 行为消息结构**</p><br>
访问 **<i>actionlib_tutorials</i>** 功能包中的 **<i>action</i>** 目录，查看Fibonacci.action消息定义的结构。

<p style="background:#EE9023;color:white;">**练习 4.12a结束**</p>

<p style="background:#EE9023;color:white;">**练习 4.12b: 观察行为服务器的反馈和结果主题的消息输出**</p><br>

重新启动python代码 <a href="#prg-4.11a">{4.11a}</a> ，运行Fibonacci服务器。
<br><br>
接着，在相应的WebShell执行下面的命令。<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">在WebShell #1运行: Echo the result topic</p>
</th>
</tr>
</table>

In [ ]:
rostopic echo /fibonacci_as/result

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">在WebShell #2运行: Echo the feedback topic</p>
</th>
</tr>
</table>

In [ ]:
rostopic echo /fibonacci_as/feedback

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">在WebShell #3运行: Manually send the goal to your Fibonacci server, publishing directly to the topic (as you learned in the previous chapter)</p>
</th>
</tr>
</table>

In [ ]:
rostopic pub /fibonacci_as/goal actionlib_tutorials/FibonacciActionGoal [TAB][TAB]

<p style="background:#AE0202;color:white;">**练习 4.12b预期结果**</p><br>
调用一个行为后，反馈主题将发布反馈，并在计算完成后发布结果。

<br>

<p style="background:#3B8F10;color:white;">**练习 4.12b数据**</p><br>
<ul>
<li>
你必须注意，Python代码中使用的消息（类）名称分别为 **<i>FibonacciGoal</i>**, **<i>FibonacciResult</i>**, 和 **<i>FibonacciFeedback</i>**, 而在主题中使用的消息名称分别为 **<i>FibonacciActionGoal</i>**, **<i>FibonacciActionResult</i>**, 和 **<i>FibonacciActionFeedback</i>**。 
<br><br>
不用担心这一点，只需记住它并相应地使用它就行。
</li>
</ul>

<p style="background:#EE9023;color:white;">**练习4.12b结束**</p>

<p style="background:#EE9023;color:white;" id="ex-4-13">**练习 4.13: 使用行为服务器创建一个控制无人机绕正方形运动的功能包**</p>
<br>
a) 使用行为服务器创建一个功能包，使无人机在被调用时绕正方形轨迹运动。
<br><br>
b) 通过主题调用行为服务器，观察结果和反馈。
<br><br>
c) 使用前面Fibonacci例子中的代码 <a href="#prg-4.11a">{4.11a}</a> 和你在练习 4.6中创建的客户端控制无人机在拍照的同时运动。
<br>

<p style="background:#AE0202;color:white;">**练习 4.13预期结果**</p><br>
如下面动画所示，当行为服务器被调用时，无人机在空中绕正方形飞行。<a href="#fig-4.6">{图:4.6}</a>
<br>

<p style="background:#3B8F10;color:white;">**练习 4.13数据**</p>
<ul>
<li>正方形每条边的大小需要在目标消息中指定，需为整数。</li>
<li>反馈应发布无人机在绕正方形运动时当前所在的边（以数字的形式）。</li>
<li>结果应发布无人机绕正方形运动的总秒数。</li>
<li>对于该行为服务器，使用 <span style="color: orange"><i>Test.action</i></span> 消息。使用内核命令 <span style="color: orange"><i>locate Test.action</i></span> 查看消息是在哪里进行定义的。接着，分析msg消息的结构以便了解如何在你的行为服务器中使用它。</li>
</ul>

<p style="background:#EE9023;color:white;" id="ex-4-13">**练习 4.13结束**</p>

<p style="background:green;color:white;">练习 4.13解决方案</p>

请尽量独自完成，除非你陷入困境或需要一些灵感。如果每一个练习你都认真对待的话，你会学到更多！

<img src="img/robotignite_logo_text.png"/>

按照下面的链接打开解决方案笔记本:[Actions Part2 Solutions](extra_files/zh_unit4_basicROS_part2_solutions.ipynb)

<p style="background:green;color:white;">练习 4.13解决方案结束</p>




<figure>
  <img id="fig-4.6" src="img/drone.gif">
  <br>
   <center> <figcaption>图.4.6 - 无人机根据一个定制的行为服务器发布的命令进行运动 Ex 4.13</figcaption></center>
</figure>

## 如何创建你自己的行为服务器消息

<b>我们始终推荐你使用ROS提供的行为消息。</b> 这些消息可以在下面的ROS功能包中找到：

* actionlib
    
 * Test.action
 * TestRequest.action 
 * TwoInts.action
    
* actionlib_tutorials
    
 * Fibonacci.action
 * Averaging.action
    

<br>

然而，有些时候，你需要创建自己的行为消息。下面将告诉你怎么做。

要创建自己的行为消息，需要按照下面的步骤来做：

1.- 在你的功能包中创建一个 **action** 目录。
  
2.- 创建你自己的 **Name.action** 行为消息文件。
  
      
 * 行为消息文件的名称将决定后面**action server**和/或**action client** 中使用的类的名称。
        
 * 注意Name.action 文件包括三部分，每一部分用三个连字符分开。

In [ ]:
#goal
package_where_message_is/message_type goal_var_name
---
#result
package_where_message_is/message_type result_var_name
---
#feedback
package_where_message_is/message_type feedback_var_name

* 如果你不需要消息中的某一部分 (例如，你不需要提供反馈）, 那么你可以使该项空着。但你 <b>必须指定分隔符</b>。    

3.- 修改CMakeLists.txt和package.xml文件，包含行为消息编译。请阅读下面的详细描述。 

<div id="custom_compilation"></div>

## 如何为定制行为消息编译修改CMakeLists.txt和package.xml 文件？

你必须按照我们在主题和服务单元介绍的方式对这两个文件进行编辑：
* CMakeLists.txt
* package.xml


### 修改 CMakeLists.txt

编辑CMakeLists.txt中的这些函数：

* <span class="ign_green"><a href="#find_package">find_package()</a></span>
* <span class="ign_green"><a href="#add_action_files">add_action_files()</a></span>
* <span class="ign_green"><a href="#generate_messages">generate_messages()</a></span>
* <span class="ign_green"><a href="#catkin_package">catkin_package()</a></span>



### <span style="color: green;" id="find_package">I. find_package()</span>

所有编译主题消息、服务消息和行为消息需要的功能包都在这里。
<br>
In <i>package.xml</i>, you have to state them as built.

In [ ]:
find_package(catkin REQUIRED COMPONENTS
      # 你的功能包在这里列出
      actionlib_msgs
)

### <span style="color: green;" id="add_action_files">II. add_action_files()</span>

该函数包含所有来自需要编译的功能包(存储在 <b>action</b> 文件夹中)的行为消息。<br>
把它们放置在FILES标签的下面。

In [ ]:
add_action_files(
      FILES
      Name.action
)

### <span style="color: green;" id="generate_messages">III. generate_messages()</span>

行为消息编译需要的功能包导入到这里。把与find_package中相同的内容添加在这里。

In [ ]:
generate_messages(
      DEPENDENCIES
      actionlib_msgs 
      # Your packages go here
)

### <span style="color: green;" id="catkin_package">IV. catkin_package()</span>

这里给出运行你功能包中的某个程序所需要的所有功能包。 这里给出的所有功能包必须作为 <run_depend> 位于 package.xml 文件中。

In [ ]:
catkin_package(
      CATKIN_DEPENDS
      rospy
      # 你的功能包依赖项
)

完成后的代码应该是这样的：

In [ ]:
cmake_minimum_required(VERSION 2.8.3)
project(my_custom_action_msg_pkg)

## Find catkin macros and libraries
## if COMPONENTS list like find_package(catkin REQUIRED COMPONENTS xyz)
## is used, also find other catkin packages
find_package(catkin REQUIRED COMPONENTS
  std_msgs 
  actionlib_msgs
)

## Generate actions in the 'action' folder
add_action_files(
   FILES
   Name.action
 )

## Generate added messages and services with any dependencies listed here
generate_messages(
   DEPENDENCIES
   std_msgs actionlib_msgs
 )

catkin_package(
 CATKIN_DEPENDS rospy
)

## Specify additional locations of header files
## Your package locations should be listed before other locations
# include_directories(include)
include_directories(
  ${catkin_INCLUDE_DIRS}
)

### 修改 package.xml:

1.- 添加编译服务消息所需要的所有功能包。

如果，**.action** 文件中的一个变量使用一个在std_msgs功能包外定义的消息，例如**nav_msgs/Odometry** ，那么你需要导入它。要这样做，你需要添加如下代码，将**nav_msgs** 功能包导入为 **&lt;build_depend&gt;** ：

In [ ]:
<build_depend>nav_msgs<build_depend>

2.- 另一方面，如果运行功能包中的程序需要一个功能包，你必须将那个需要的功能包导入为 **&lt;run_depend&gt;** 。添加如下代码：

In [ ]:
<run_depend>nav_msgs</run_depend>

当你编译定制行为消息时，需要**强制** 将**actionlib_msgs** 添加为build_dependency。

In [ ]:
<build_depend>actionlib_msgs</build_depend>

如果你使用的是Python，需要 **强制** 将 **rospy** 作为 **run_dependency** 。

In [ ]:
<run_depend>rospy</run_depend>

这是由于运行所有的python ROS代码需要rospy python模块。

概括起来，最后的package.xml文件看起来是这样的：

In [ ]:
<?xml version="1.0"?>
<package>
  <name>my_custom_action_msg_pkg</name>
  <version>0.0.0</version>
  <description>The my_custom_action_msg_pkg package</description>
  <maintainer email="user@todo.todo">user</maintainer>
  <license>TODO</license>

  <buildtool_depend>catkin</buildtool_depend>
  <build_depend>std_msgs</build_depend>
  <build_depend>actionlib_msgs</build_depend>
  
  <run_depend>rospy</run_depend>

  <export>
  </export>
</package>

最后，一切设置正确后，运行下面的命令进行编译：

In [ ]:
roscd; cd ..
catkin_make
source devel/setup.bash
rosmsg list | grep Name

你会得到类似下面的输出：

In [ ]:
my_custom_action_msg_pkg/NameAction
my_custom_action_msg_pkg/NameActionFeedback
my_custom_action_msg_pkg/NameActionGoal
my_custom_action_msg_pkg/NameActionResult
my_custom_action_msg_pkg/NameFeedback
my_custom_action_msg_pkg/NameGoal
my_custom_action_msg_pkg/NameResult

<p style="background:green;color:white;">**注**</p><br>
注意，你没有在所有的地方都导入<b>std_msgs</b>功能包。但你可以在你的定制行为中使用该功能包中声明的消息。这是由于该功能包是roscore文件系统的一部分， 它嵌入到了编译协议中，因此无需再进行声明。

<p style="background:#EE9023;color:white;">**练习 4.14: 使用定制行为消息的行为服务器创建一个移动无人机的功能包**</p>

* 在前面的功能包中的，你创建了 <a href="#ex-4-13">[Ex 4.13]</a>, 为四旋翼无人机创建了一个新的行为服务器。
* 新的行为服务器将接收两个单词作为目标： UP 或者 DOWN。
* 当行为服务器接收到 UP 时, 它将使无人机向上运动1米。
* 当行为服务器接收到 DOWN 时, 它将使无人机向下运动1米。
* 作为反馈，它每秒反馈一次正在发生的行为（向上或向下）。
* 行为结束后，什么都不返回。

<p style="background:#3B8F10;color:white;">**练习 4.14数据**</p>

* 你需要创建一个类型为<i>String</i>的新行为消息。该类型可以从 <i>std_msgs</i>功能包导入。
* 该行为消息的结果部分将是空的。
* 由于我们用的是无人机，你可以在三个轴上指定扭转（Twist）速度。这样做可以上下移动无人机。

<p style="background:#EE9023;color:white;">**练习 4.14结束**</p>

<p style="background:green;color:white;">练习 4.14解决方案</p>

请尽量独自完成，除非你陷入困境或需要一些灵感。如果每一个练习你都认真对待的话，你会学到更多！

<img src="img/robotignite_logo_text.png"/>

按照下面的链接打开解决方案笔记本：[Actions Part2 Solutions](extra_files/unit4_basicROS_part2_solutions.ipynb)

<p style="background:green;color:white;">练习 4.14解决方案结束</p>

## 更多的学习资料

ROS 行为: http://wiki.ros.org/actionlib

行为工作机制: http://wiki.ros.org/actionlib/DetailedDescription